In [6]:
import pygame
import os
import time
import random
import neat
pygame.font.init()


WIDTH, HEIGHT = 600, 800

WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flappy Bird")

path = 'C:\\Users\\BaDWoLF4096\\Desktop\\Flappy Bird'

BASE =  pygame.transform.scale(pygame.image.load(os.path.join(path, 'imgs', 'base.png')),(WIDTH, 200))
BG = pygame.transform.scale(pygame.image.load(os.path.join(path, 'imgs', 'bg.png')),(WIDTH, HEIGHT))

BIRD_IMG = [pygame.image.load(os.path.join(path, 'imgs', 'bird1.png')), pygame.image.load(os.path.join(path, 'imgs', 'bird2.png')), pygame.image.load(os.path.join(path, 'imgs', 'bird3.png'))]

PIPE_IMG = pygame.image.load(os.path.join(path, 'imgs', 'pipe.png'))

class Bird:
    IMG = BIRD_IMG
    MAX_ROTATION = 25
    ROT_VEL = 20
    ANIMATION_TIME = 5
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.tilt = 0
        self.vel = 0
        self.height = self.y
        self.img_count = 0
        self.img = BIRD_IMG[1]
        self.mask = pygame.mask.from_surface(self.img)
        
    def draw(self, window):
        window.blit(self.img, (self.x, self.y))
        
    def jump(self):
        self.vel = 10
        self.height = self.y
        self.img = BIRD_IMG[1]
        self.y -= 20
    
    def up(self):
        self.img = BIRD_IMG[0]
        pygame.display.update()
        
    def down(self):
        self.img = BIRD_IMG[2]
        pygame.display.update()
        
    def gravity(self):
        self.y += 6
        
    def collision(self, obj):
        return collide(obj, self)
        
        
class bg_stuff:
    def __init__(self, x=0, y=700):
        self.x = x
        self.y = y
        self.x2 = BASE.get_width()
        self.img = [BASE, BASE]
    
    
    def draw(self):
        WIN.blit(self.img[0], (self.x, self.y))
        WIN.blit(self.img[1], (self.x2, self.y))
        
    
    def move(self):
        self.x -= 3
        self.x2 -= 3
        
        if self.x + self.get_width() < 0:
            self.x = self.x2 + self.get_width()
        if self.x2 + self.get_width() < 0:
            self.x2 = self.x + self.get_width()
            
        self.draw()
        
        
    def get_width(self):
        return self.img[0].get_width()
    
    def get_height(self):
        return self.img[0].get_height()
    
class Pipe:
    def __init__(self, x, y, angle=0):
        self.x = x
        self.y = y
        self.pipe_img = pygame.transform.rotate(PIPE_IMG, angle)
        self.mask = pygame.mask.from_surface(self.pipe_img)
    
    def draw(self):
        WIN.blit(self.pipe_img, (self.x, self.y))
    
    def get_width(self):
        return self.pipe_img.get_width()
    
    def get_height(self):
        return self.pipe_img.get_height()
    
def collide(obj1, obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return obj1.mask.overlap(obj2.mask, (offset_x, offset_y)) !=  None
    

def main(genomes, config):
    nets = []
    ge = []
    birds = []
    
    for g in genomes:
        net = neat.nn.FeedForwardNetwork(g, config)
        nets.append(net)
        birds.append(Bird(300, 400))
        g.fitness = 0
        ge.append(g)
    
    #bird = Bird(300, 400)
    base = bg_stuff()
    run = True
    FPS = 60
    clock = pygame.time.Clock()
    start = False
    main_font = pygame.font.SysFont("comicsans", 50)
    lost_font = pygame.font.SysFont("comicsans", 80)
    pipes = []
    temp = []
    pipes_temp = []
    score = 0
    def redraw_window():
        WIN.blit(BG, (0,0))
        score_label = main_font.render(f"Score:{int(score)}", 1, (255,255,255))
        
        WIN.blit(score_label, (10,10))
        
        bird.draw(WIN)
        for pipe in pipes:
            pipe.x -= 3
            pipe.draw()
            
        if run == False:
            lost = lost_font.render(f"Lost", 1, (255,255,255))
            WIN.blit(lost, (WIDTH/2 - lost.get_width()/2, HEIGHT/2))
        
        base.move()
        
        
        pygame.display.update()
    
    
    while run:
        clock.tick(FPS)
        #bird.move()
        
        
        if len(pipes) == 0:
            
            for i in range(5):
                x = random.randrange(800 , 3000, 200)
                if x in temp:
                    x += 200
                temp.append(x)
                y = random.randrange(400, 500)
                pipe_down = Pipe(x, y, 0)
                pipe_up = Pipe(x, y - pipe_down.get_height() - 200, 180)
                pipes.append(pipe_up)
                pipes.append(pipe_down)
                
            pipes_temp = pipes.copy()
        
        
        #base.x -= 3 
        #base.x = base.x % (base.get_width()*-1)
           
        
        for bird in birds:
            for pipe in pipes_temp:
                if bird.x > pipe.x + pipe.get_width()/2:
                    score += 1/2
                    pipes_temp.remove(pipe) 

        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit()
        
        for bird in birds:
            if bird.y < 670:
                pass
        
        
            for pipe in pipes:
                if bird.collision(pipe):
                    run = False
                if pipe.x + pipe.get_width() < 0:
                    pipes.remove(pipe)
                
        
        keys = pygame.key.get_pressed()
        if keys[pygame.K_w]:
            start = True
            bird.up()
            bird.jump()
            bird.down()
            
        elif start:    
            bird.gravity()
        
        

        redraw_window()
main()



In [ ]:
def run(config_file):
    
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #p.add_reporter(neat.Checkpointer(5))

    # Run for up to 50 generations.
    winner = p.run(eval_genomes, 50)

    # show final stats
    print('\nBest genome:\n{!s}'.format(winner))


if __name__ == '__main__':
    local_dir = os.path.dirname(__file__)
    config_path = os.path.join(local_dir, 'config-feedforward.txt')
    run(config_path)

In [ ]:

[NEAT]
fitness_criterion     = max
fitness_threshold     = 100
pop_size              = 50
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = tanh
activation_mutate_rate  = 0.0
activation_options      = tanh

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 3
num_outputs             = 1

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2

In [ ]:
nets = []
    ge = []
    birds = []
    
    for g in genomes:
        net = neat.nn.FeedForwardNetwork(g, config)
        nets.append(net)
        birds.append(Bird(300, 400))
        g.fitness = 0
        ge.append(g)
    ''''